In [1]:
from zeno_client import ZenoClient, ZenoMetric
import pandas as pd
import os
import dotenv

dotenv.load_dotenv(override=True)

False

In [2]:
OUTPUT_DIR = "../../gemini-benchmark/outputs/gsm8k"

In [3]:
models = os.listdir(OUTPUT_DIR)

In [4]:
models.remove(".ipynb_checkpoints")

In [5]:
# Upload base dataset
df = pd.read_json(os.path.join(OUTPUT_DIR, models[0], "output.jsonl"), lines=True)
base_df = pd.DataFrame({
    "qid": df["qid"],
    "question": df["question"],
    "answer": df.apply(lambda x: x['answer_text'].split("<|endoftext|>")[0], axis=1) 
})

In [6]:
zeno_client = ZenoClient("zen_P4CK880bWHV2dJLbAenb0r8Gf6QNLdTXhSkkHqPDm4I")

/data/tir/projects/tir4/users/sakter/anaconda3/envs/bleed/lib/python3.9/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package zeno-client is out of date. Your version is 0.1.13, the latest is 0.1.14.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [7]:
project = zeno_client.create_project(
    name="Gemini Evaluation - GSM8k",
    description="Evaluation of Gemini, GPT-4, and Mixtral on GSM8k dataset",
    view={
        "data": {
            "type": "markdown"
        },
        "label": {
            "type": "markdown"
        },
        "output": {
            "type": "markdown"
        }
    },
    public=True,
    metrics=[
        ZenoMetric(name="Accuracy Strict Match", type="mean", columns=["is_correct"]),
        ZenoMetric(name="Accuracy", type="mean", columns=["is_correct_last"])
    ],
)

Successfully updated project.
Access your project at  https://hub.zenoml.com/project/1506e92b-a305-499c-aad1-d78f6f47bff0/Gemini%20Evaluation%20-%20GSM8k


In [8]:
project.upload_dataset(base_df, id_column="qid", data_column="question", label_column="answer")

  0%|          | 0/1 [00:00<?, ?it/s]

Successfully uploaded data


In [14]:
def check_fraction(num):
    print(num)
    if num % 1 != 0:
        return 1
    return 0

for model in models:
    df = pd.read_json(os.path.join(OUTPUT_DIR, model, "output.jsonl"), lines=True)
    if model == 'mixtral':
        output_df = pd.DataFrame({
            "qid": df["qid"],
            "output": df.apply(lambda x: f"{x['generated_text']}\n\n**{x['predict']}**", axis=1),
            "answer_str" : df.apply(lambda x: f"{str(x['answer'])}", axis=1), 
            "fraction": df.apply(lambda x: check_fraction(float(x['answer'])), axis=1),
            "numeric_answer": df.apply(lambda x: float(x['answer']), axis=1),
            "question_length": df.apply(lambda x: len(x['question'].split(' ')), axis=1),
            "label_length": df.apply(lambda x: len(x['answer_text'].split(' ')), axis=1),
            "output_length": df.apply(lambda x: len(x['generated_text'].split('Q:')[0].split(' ')), axis=1),
            "is_correct": df["is_correct"].astype(bool),
            "is_correct_last": df["is_correct_last"].astype(bool)
        })
    else:
        output_df = pd.DataFrame({
            "qid": df["qid"],
            "output": df.apply(lambda x: f"{x['generated_text']}\n\n**{x['predict']}**", axis=1),
            "answer_str" : df.apply(lambda x: f"{str(x['answer'])}", axis=1), 
            "fraction": df.apply(lambda x: check_fraction(float(x['answer'])), axis=1),
            "numeric_answer": df.apply(lambda x: float(x['answer']), axis=1),
            "question_length": df.apply(lambda x: len(x['question'].split(' ')), axis=1),
            "label_length": df.apply(lambda x: len(x['answer_text'].split(' ')), axis=1),
            "output_length": df.apply(lambda x: len(x['generated_text'].split(' ')), axis=1),
            "is_correct": df["is_correct"].astype(bool),
            "is_correct_last": df["is_correct_last"].astype(bool)
        })
    if model == 'gpt-4-1106-preview':
        model = 'gpt-4-turbo'
    project.upload_system(output_df, name=model, id_column="qid", output_column="output")

2800.0
20.0
10.0
4.0
15.0
25.0
560.0
90.0
6.0
26.0
70.0
5.0
8.0
68.0
1430.0
132.0
175.0
91.0
6.0
82.0
129200.0
600.0
7.0
288.0
1.0
20.0
240.0
20.0
50.0
2.0
2.0
5.0
4.0
36.0
40.0
5.0
79.0
310.0
20.0
17.0
78.0
43500.0
240.0
300.0
145.0
539.0
8.0
95200.0
296.0
36.0
525.0
34.0
100.0
44.0
12.0
12.0
3.0
75.0
350.0
25.0
7400.0
80.0
52.0
2.0
70.0
5.0
8.0
18.0
160.0
4.0
16.0
224000.0
260.0
720.0
130.0
34.0
8.0
13.0
50.0
480.0
100.0
40.0
70.0
45.0
120.0
40.0
45.0
27.0
10.0
60.0
6.0
133.0
54.0
6.0
11.0
452.0
694.0
8.0
9500.0
200.0
736.0
60.0
7.0
192.0
9.0
172.0
30.0
770.0
45.0
480.0
64.0
18000.0
215.0
70000.0
131250.0
9.0
8.0
300.0
22.0
576.0
10.0
2400.0
68.0
45.0
10.0
27000.0
15.0
250.0
350.0
13.0
93000.0
87.0
49.0
20.0
210.0
8.0
4.0
144.0
43200.0
200.0
14000.0
10.0
2.0
7.0
6.0
40.0
8.0
12.0
4400.0
8.0
16.0
18.0
34.0
10.0
90.0
28.0
140.0
15.0
10.0
27.0
10.0
360.0
138.0
48.0
39.0
71.0
2304.0
540.0
3.0
2880000.0
20.0
50.0
272.0
113.0
3.0
168.0
26.0
30.0
10.0
6400.0
52.0
3.0
66.0
315.0
60000.0
294.

  0%|          | 0/2 [00:00<?, ?it/s]

Successfully uploaded system
91.0
175.0
2.0
720.0
4.0
60.0
623.0
80.0
45.0
3360.0
9.0
168000.0
940.0
18.0
270.0
4000.0
120.0
36.0
23.0
120.0
21.0
11.0
120.0
38.0
10.0
8.0
11.0
80.0
15.0
16.0
131250.0
40.0
6.0
350.0
30.0
75.0
7.0
7.0
4.0
500.0
1875.0
9.0
25.0
8.0
40.0
255.0
75.0
35.0
14.0
48.0
7300.0
25.0
96.0
2280.0
7500.0
6.0
150.0
25.0
10.0
15.0
291.0
105.0
60.0
4500.0
247.0
4.0
29.0
525.0
7.0
800.0
2.0
14.0
36.0
750.0
35.0
31.0
3.0
1125.0
3200.0
8.0
4.0
390.0
360.0
11.0
101.0
8.0
40.0
1.0
172.0
5600.0
71.0
240.0
69.0
20.0
10.0
4.0
1200.0
1600.0
18.0
144.0
75.0
16.0
10.0
250.0
50.0
4.0
30.0
16.0
7.0
120.0
2325.0
14.0
85000.0
8.0
64.0
10.0
10.0
50.0
110.0
1.0
15.0
1100.0
60.0
16.0
18.0
10.0
15.0
36.0
57500.0
880.0
5.0
10.0
44.0
16.0
38.0
93000.0
10.0
76.0
1080.0
4.0
60.0
24.0
100.0
140.0
9.0
13.0
157.0
17.0
12.0
36.0
48.0
1596.0
160.0
125.0
2000.0
48.0
20.0
665.0
10.0
15.0
7.0
800.0
26.0
350.0
29.0
5.0
410.0
10.0
5.0
300.0
20.0
36.0
25.0
3.0
54.0
6.0
24.0
2880000.0
240.0
6.0
32.0
3430

  0%|          | 0/3 [00:00<?, ?it/s]

Successfully uploaded system
85.0
9.0
24.0
18.0
13.0
5.0
140.0
40000.0
2050.0
7.0
40.0
1430.0
6.0
91.0
5.0
5.0
80.0
9.0
3.0
80.0
3.0
57500.0
700.0
12.0
48.0
2.0
25.0
50.0
342.0
280.0
34.0
300.0
120.0
50.0
1100.0
3.0
2.0
30.0
101.0
81.0
20.0
385000.0
2400.0
15.0
75.0
14.0
45.0
8.0
2600.0
17.0
4.0
44.0
142.0
20.0
8400.0
53.0
75.0
23.0
106.0
19.0
63.0
225.0
250.0
83.0
15.0
125.0
45.0
168.0
4.0
1200.0
77.0
25.0
1.0
122.0
8.0
2.0
40.0
3.0
5.0
4000.0
31.0
80.0
100.0
9.0
700.0
72000.0
5.0
17.0
2400.0
4.0
100.0
975.0
300.0
142.0
54.0
21.0
3.0
55.0
350.0
3000.0
2800.0
24.0
26.0
193.0
720.0
48.0
14.0
23.0
6.0
655.0
5.0
600.0
1596.0
8.0
60.0
240.0
4.0
2.0
75.0
276000.0
37.0
8.0
195.0
14000.0
79.0
34.0
69.0
1509.0
5600.0
192.0
4.0
77.0
153.0
15.0
180.0
5.0
310.0
480.0
9.0
260.0
1.0
10.0
32.0
67.0
6250.0
1.0
16.0
11.0
460.0
880.0
1300.0
104.0
70.0
30.0
2.0
144.0
109.0
50.0
5.0
14.0
12.0
6.0
6.0
60000.0
48.0
19.0
19.0
49.0
4500.0
21.0
162000.0
100.0
2.0
1800.0
94.0
225.0
33.0
1248.0
32.0
5.0
6.0
8.0

  0%|          | 0/1 [00:00<?, ?it/s]

Successfully uploaded system
60.0
13.0
720.0
40.0
6.0
29.0
105.0
70.0
20.0
400.0
140.0
16.0
20.0
4000.0
2125.0
75.0
30.0
16.0
4.0
5.0
4.0
48.0
272.0
280.0
1400.0
80.0
34.0
15.0
16.0
32.0
92.0
50.0
15.0
77.0
5.0
16.0
18.0
120.0
150.0
1210.0
51.0
18000.0
95.0
15.0
100.0
350.0
122.0
130.0
20.0
160.0
23.0
2.0
25.0
30.0
5.0
106.0
50.0
34.0
360.0
5.0
91.0
24.0
10.0
12.0
120.0
6277.0
320.0
7500.0
55.0
114200.0
100.0
31.0
98.0
98.0
860.0
2600.0
76.0
145.0
10.0
4.0
5.0
250.0
8.0
44.0
220.0
15.0
45.0
54.0
70.0
90.0
140.0
20000.0
180.0
9.0
33.0
9.0
1.0
21.0
276000.0
50.0
75.0
12.0
21.0
10.0
31.0
90.0
68.0
280.0
21.0
6.0
3.0
250.0
20.0
7.0
27000.0
32.0
300.0
5600.0
17.0
70.0
73.0
18.0
84.0
192.0
45.0
5600.0
6.0
168.0
11.0
62.0
270.0
8.0
400.0
9500.0
118000.0
91.0
1375.0
4.0
762.0
20.0
5.0
315.0
3200.0
138.0
9.0
4.0
40.0
6.0
7.0
2450.0
195.0
68.0
360.0
21.0
90.0
8.0
3.0
16.0
390.0
2.0
75.0
83.0
3.0
370.0
3.0
55.0
500.0
31800.0
78.0
8.0
15.0
1300.0
3200.0
4.0
10.0
16.0
6.0
8.0
2050.0
91.0
32.0
12000

  0%|          | 0/1 [00:00<?, ?it/s]

Successfully uploaded system
